<a href="https://colab.research.google.com/github/rubanzasilva/random_forest_basics/blob/main/random_forests_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#hide
#This imports and sets up everything you will need for this notebook
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00
Mounted at /content/gdrive


In [1]:
from fastai.imports import *
np.set_printoptions(linewidth=130)

# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path
import os

Data PreProcessing

In [4]:
creds = '{"username":"rubanzasilva","key":"7590231111c5bbe2ad242827b0ef016c"}'

In [5]:
creds = '{"username":"rubanzasilva","key":"7590231111c5bbe2ad242827b0ef016c"}'

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)


In [6]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle: path = Path('../input/titanic')
else:
    import zipfile,kaggle
    path = Path('titanic')
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')

100%|██████████| 34.1k/34.1k [00:00<00:00, 8.23MB/s]

Preprocessing

We do the same processing we had done in our previous linear model.

With random forests, we dont need to create dummy variables instead we turn our variables into categorical which assigns each level in a column a number/label.The number is an index for looking up that particular value in a list of unique values.

Lets create a function for our transformations.

In [20]:
modes = train_df.mode().iloc[0]
modes

PassengerId                      1
Survived                       0.0
Pclass                         3.0
Name           Abbing, Mr. Anthony
Sex                           male
Age                           24.0
SibSp                          0.0
Parch                          0.0
Ticket                        1601
Fare                          8.05
Cabin                      B96 B98
Embarked                         S
LogFare                   2.202765
Name: 0, dtype: object

In [10]:
def proc_data(df):
  df['Fare'] = df.Fare.fillna(0)
  df.fillna(modes,inplace=True)
  df['LogFare'] = np.log1p(df['Fare'])
  df['Sex'] = pd.Categorical(df.Sex)
  df['Embarked'] = pd.Categorical(df.Embarked)

In [14]:
proc_data(train_df)
proc_data(test_df)

In [16]:
train_df.shape

(891, 13)

Previously we had defined sex, embarked, and pclass as our categorical variables. We turn these into categorical variables here which for every value, assigns each value a number within a list of unique values.

We also notice that Pclass is an ordered variable, in decision trees we see that the order of classes carries some significant information about the data. This means that the decision tree can make decisions based on the order of our variables and not just the distinct values.

The model takes into account the order and hierarchy in the data which leads to more accurate predictions.

In [26]:
cats = ['Embarked', 'Sex']
conts = ['Pclass','Age','SibSp','Parch','LogFare']
dep = 'Survived'